<a href="https://colab.research.google.com/github/dogakusun/Yemek_Tarifleri_RAG_Chatbotu/blob/main/Yemek_Tarifleri_RAG_Chatbotu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Gerekli kütüphaneleri kurma
!pip install -q google-genai langchain langchain-community chromadb streamlit datasets pypdf

In [32]:
!pip install langchain-google-genai

In [33]:
import os
from google.colab import userdata

# Colab'den GEMINI_API_KEY'i al ve ortam değişkeni olarak ayarla
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

print("Ortam Hazırlığı Tamamlandı. API Anahtarı Yüklendi.")

Ortam Hazırlığı Tamamlandı. API Anahtarı Yüklendi.


In [34]:
# -----------------------------------------------------------
# 2.1. Veri Setini Yükleme ve Yapılandırma (TEMİZLEME EKLENDİ)
# -----------------------------------------------------------
from datasets import load_dataset
from langchain_core.documents import Document

print("Veri Seti Yükleniyor ve Temizleniyor...")

dataset = load_dataset("mertbozkurt/llama2-TR-recipe", split="train")
recipes = dataset.select(range(5000))

langchain_docs = []
for i, item in enumerate(recipes):
    recipe_text = item.get('text', "")

    if not recipe_text:
         continue

    # KRİTİK TEMİZLEME ADIMI: Etiketleri kaldırıyoruz
    recipe_text = recipe_text.replace("<s>[INST]", "SORU:").replace("[/INST] ", "CEVAP:").replace("</s>", "")

    doc = Document(
        page_content=recipe_text,
        metadata={"source": "HuggingFace Recipe Dataset", "recipe_id": i}
    )
    langchain_docs.append(doc)

print(f"Toplam {len(langchain_docs)} adet tarif belgesi temizlenerek oluşturuldu.")



Veri Seti Yükleniyor ve Temizleniyor...
Toplam 5000 adet tarif belgesi temizlenerek oluşturuldu.


In [35]:
# -----------------------------------------------------------
# 2.2. Metin Parçalama
# -----------------------------------------------------------
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Bu kodun altındaki 2.2 Metin Parçalama kodunu aynen tekrar çalıştırın.
# (chunks listesini yeni, temizlenmiş verilerle yeniden doldurmak için)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(langchain_docs)
print(f"Parçalama tamamlandı: {len(chunks)} adet parça.")

Parçalama tamamlandı: 5000 adet parça.


In [36]:
# -----------------------------------------------------------
# 2.3. Embedding ve ChromaDB Oluşturma
# -----------------------------------------------------------
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
import os
from google.colab import userdata

print("Embedding ve Vektör Veritabanı (ChromaDB) Oluşturuluyor... Bu adım zaman alabilir.")

# API Anahtarını doğrudan Colab Gizli Anahtarlarından çekme
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# Google'ın text-embedding-004 modelini kullanma
embeddings = GoogleGenerativeAIEmbeddings(
    model="text-embedding-004",
    google_api_key=GEMINI_API_KEY
)

# Veritabanını Colab ortamında kalıcı bir klasöre kaydetme.
persist_dir = "./chroma_db_tarifler"

# ChromaDB'yi oluştur ve verileri gömerek kaydetme

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)

# Arayüz için bir retriever (geri çağırıcı) hazırla
# k=6 olarak yükseltildi: En alakalı 6 metin parçasını çekecek
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

print("-" * 50)
print(f"ChromaDB başarıyla oluşturuldu ve '{persist_dir}' dizinine kaydedildi.")
print("Retriever k=6 olarak güncellendi.")

Embedding ve Vektör Veritabanı (ChromaDB) Oluşturuluyor... Bu adım zaman alabilir.
--------------------------------------------------
ChromaDB başarıyla oluşturuldu ve './chroma_db_tarifler' dizinine kaydedildi.
Retriever k=6 olarak güncellendi.


In [37]:
# -----------------------------------------------------------
# 3.1. LangChain RAG Zincirini Kurma
# -----------------------------------------------------------
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

print("RAG Zinciri Bileşenleri Tanımlanıyor...")

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

# 1. LLM'i Tanımlama: SİSTEM TALİMATI EKLENDİ ve TEMPERATURE 0.0
# LLM'i, prompt yerine doğrudan model seviyesinde yönlendiriyoruz.
system_prompt_instruction = (
    "Sen, yalnızca sana sağlanan BAĞLAM (Context) üzerinden cevap veren, Türk mutfağına özel bir asistansın. "
    "Bağlamda bilgi varsa, bu bilgiyi KESİNLİKLE kullanmak ve cevap üretmek zorundasın. Cevap formatı sadece Yemek Adı, Malzemeler, Tarif/Yapılışı olmalıdır. "
    "Eğer bağlamda cevap verecek hiçbir bilgi yoksa, sadece şu cümleyi kullan: 'Üzgünüm, aradığınız tarifi şu anda veri tabanımda bulamıyorum. Belki başka bir yemek tarifi sormak istersiniz?'"
)

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    google_api_key=GEMINI_API_KEY,
    # SİSTEM TALİMATINI DOĞRUDAN MODEL SEVİYESİNDE GÖNDERİYORUZ
    model_kwargs={"system_instruction": system_prompt_instruction}
)

# 2. Prompt Şablonu (Artık LLM yönlendirildiği için bu prompt SADECE formatı verecek)
RAG_PROMPT = """
Aşağıdaki BAĞLAM'ı kullanarak SORU'ya cevap ver. Cevabını **Yemek Adı**, **Malzemeler** ve **Yapılışı** başlıkları altında düzenle.

BAĞLAM:
{context}

SORU:
{question}
"""
prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

# 3. RAG Zincirini Birleştirme
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("-" * 50)
print("RAG Zinciri Başarıyla KURULDU. Sistem Talimatı ile LLM zorunlu yönlendirildi.")

RAG Zinciri Bileşenleri Tanımlanıyor...
--------------------------------------------------
RAG Zinciri Başarıyla KURULDU. Sistem Talimatı ile LLM zorunlu yönlendirildi.


In [38]:
# -----------------------------------------------------------
# 3.2. RAG Zincirini Test Etme
# -----------------------------------------------------------
print("RAG Zinciri Test Ediliyor...")

# 1. Test Sorgusu
question_1 = "Sodalı Köfte nasıl yapılır?"
print(f"\n[TEST 1] SORU: {question_1}")

# rag_chain'i çağır
response_1 = rag_chain.invoke(question_1)

print("-" * 20 + " CEVAP " + "-" * 20)
print(response_1)

print("\n\n" + "=" * 50 + "\n")

# 2. Test Sorgusu (Başarısız olması ve reddetmesi beklenen)
question_2 = "Amerikan hot-dog tarifi nedir?"
print(f"[TEST 2] SORU: {question_2}")

# rag_chain'i çağır
response_2 = rag_chain.invoke(question_2)

print("-" * 20 + " CEVAP " + "-" * 20)
print(response_2)

RAG Zinciri Test Ediliyor...

[TEST 1] SORU: Sodalı Köfte nasıl yapılır?
-------------------- CEVAP --------------------
**Yemek Adı**
Sodalı Köfte

**Malzemeler**
*   500 gr kıyma
*   1 adet büyük boy kuru soğan
*   1/2 çay bardağı galeta unu
*   1 tatlı kaşığı tuz
*   1 çay kaşığı dolusu kırmızı toz biber
*   1 çay kaşığı kırmızı pul biber
*   1 çay kaşığı kimyon
*   1/2 çay kaşığı karabiber
*   1/2 paket kabartma tozu
*   1 çay bardağı soda

**Yapılışı**
Verilen bağlamda Sodalı Köfte'nin yapılışına dair bilgi bulunmamaktadır, sadece malzemeler listelenmiştir.



[TEST 2] SORU: Amerikan hot-dog tarifi nedir?
-------------------- CEVAP --------------------
Verilen BAĞLAM'da "Amerikan hot-dog" tarifi bulunmamaktadır. Bu nedenle sorunuzu mevcut bağlamı kullanarak yanıtlayamıyorum.


In [39]:
# -----------------------------------------------------------
# 4.1. Streamlit İçin Gerekli Paketi Kurma
# -----------------------------------------------------------
!pip install -q streamlit-colab

In [40]:
# -----------------------------------------------------------
# 4.2. `app.py` Dosyasını Oluşturma
# -----------------------------------------------------------
%%writefile app.py
# app.py içeriği
import streamlit as st
import os
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ==================== 1. BAĞLANTILARI KURMA FONKSİYONU ====================
@st.cache_resource
def setup_rag_chain():
    """RAG zincirini kurar ve API anahtarını yönetir."""

    # 1. API Anahtarını Ortam Değişkeninden Çek
    # Anahtarın notebook ortamında os.environ'a ayarlandığını varsayıyoruz.
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

    if not GEMINI_API_KEY:
         # Hata mesajını daha genel tutuyoruz
         st.error("API Anahtarı bulunamadı. Lütfen Colab Secrets ve os.environ ayarlarını kontrol edin.")
         return None

    # 1. Embedding ve Retriever'ı Yükle
    embeddings = GoogleGenerativeAIEmbeddings(
        model="text-embedding-004",
        google_api_key=GEMINI_API_KEY
    )

    # Vektör Veritabanı Yolu
    persist_dir = "./chroma_db_tarifler"
    if not os.path.exists(persist_dir):
        st.error("Veritabanı dizini bulunamadı. Lütfen 2. Adımı çalıştırarak `chroma_db_tarifler` dizinini oluşturun.")
        return None

    # Veritabanını yükle ve retriever'ı k=6 ile hazırla
    vectorstore = Chroma(
        persist_directory=persist_dir,
        embedding_function=embeddings
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

    # 2. LLM ve Prompt'u Tanımla (SİSTEM TALİMATI VE TEMPERATURE 0.0)
    system_prompt_instruction = (
        "Sen, yalnızca sana sağlanan BAĞLAM (Context) üzerinden cevap veren, Türk mutfağına özel bir asistansın. "
        "Bağlamda bilgi varsa, bu bilgiyi KESİNLİKLE kullanmak ve cevap üretmek zorundasın. Cevap formatı sadece Yemek Adı, Malzemeler, Tarif/Yapılışı olmalıdır. "
        "Eğer bağlamda cevap verecek hiçbir bilgi yoksa, sadece şu cümleyi kullan: 'Üzgünüm, aradığınız tarifi şu anda veri tabanımda bulamıyorum. Belki başka bir yemek tarifi sormak istersiniz?'"
    )

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0.0,
        google_api_key=GEMINI_API_KEY,
        model_kwargs={"system_instruction": system_prompt_instruction}
    )

    # LangChain Prompt
    RAG_PROMPT = """
    Aşağıdaki BAĞLAM'ı kullanarak SORU'ya cevap ver. Cevabını **Yemek Adı**, **Malzemeler** ve **Yapılışı** başlıkları altında düzenle.

    BAĞLAM:
    {context}

    SORU:
    {question}
    """
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

    # 3. RAG Zincirini Birleştir
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain


# ==================== 2. STREAMLIT UYGULAMASI BAŞLANGICI ====================

st.set_page_config(page_title="Yemek Tarifleri RAG Chatbot", layout="wide")
st.title("👨‍🍳 Yemek Tarifleri RAG Chatbotu")

rag_chain = setup_rag_chain()

if rag_chain:
    # Uygulama başarıyla yüklendi, artık devam edebilir
    if "messages" not in st.session_state:
        st.session_state.messages = []
        st.session_state.messages.append({"role": "assistant", "content": "Merhaba! Ben Yemek Tarifleri Asistanınız. Hangi yemeğin tarifini arıyorsunuz?"})

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    if prompt := st.chat_input("Bir tarif sorusu yazın (örneğin: 'mercimek çorbası nasıl yapılır?'):"):

        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        with st.chat_message("assistant"):
            with st.spinner("Tarif aranıyor ve üretiliyor..."):
                try:
                    full_response = rag_chain.invoke(prompt)
                    st.markdown(full_response)
                except Exception as e:
                    st.error(f"Bir hata oluştu: {e}")
                    full_response = "Teknik bir sorun nedeniyle şu an cevap veremiyorum."

        st.session_state.messages.append({"role": "assistant", "content": full_response})

Overwriting app.py


In [41]:
# -----------------------------------------------------------
# 4.3.1. Gerekli Paketi Kaldır ve Yeniden Yükle
# -----------------------------------------------------------
!pip uninstall -y streamlit-colab
!pip install -q streamlit-colab

print("Paket temizlendi ve yeniden yüklendi.")

Found existing installation: streamlit-colab 0.1.6
Uninstalling streamlit-colab-0.1.6:
  Successfully uninstalled streamlit-colab-0.1.6
Paket temizlendi ve yeniden yüklendi.


In [42]:
# -----------------------------------------------------------
# 4.3.1. ngrok Authtoken Kurulumu
# -----------------------------------------------------------
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "34EzahU9gxdPpVeWKVkWRpiEvYW_877LWRQbrbQWdrGGLzkWi"

# ngrok servisini kimlik doğrulama belirtecinizle yapılandırın
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("ngrok Authtoken başarıyla ayarlandı.")

ngrok Authtoken başarıyla ayarlandı.


In [43]:
# -----------------------------------------------------------
# 4.2.1. API Anahtarını Ortam Değişkenine Tanımlama
# -----------------------------------------------------------
import os
from google.colab import userdata

print("API Anahtarı Ortam Değişkenine Yükleniyor...")

# Colab Secrets'tan anahtarı çekme
api_key_value = userdata.get('GEMINI_API_KEY')

if api_key_value:
    # Anahtarı Streamlit'in de erişebileceği işletim sistemi ortamına yazın
    os.environ['GEMINI_API_KEY'] = api_key_value
    print("GEMINI_API_KEY, os.environ'a başarıyla atandı.")
else:
    print("HATA: Colab Secrets'ta 'GEMINI_API_KEY' anahtarı bulunamadı.")

API Anahtarı Ortam Değişkenine Yükleniyor...
GEMINI_API_KEY, os.environ'a başarıyla atandı.


In [44]:
# -----------------------------------------------------------
# 4.3. Streamlit Uygulamasını Çalıştırma
# -----------------------------------------------------------
from pyngrok import ngrok
import time
import os
from google.colab import userdata

api_key_value = userdata.get('GEMINI_API_KEY')
if not api_key_value and not os.environ.get('GEMINI_API_KEY'):
    print("UYARI: API anahtarı hala ayarlı değil gibi görünüyor. Önceki adımı kontrol edin.")

# 1. Streamlit uygulamasını arka planda başlat
# &>/dev/null& komutu çıktıyı gizler ve süreci arka plana atar
!streamlit run app.py &>/dev/null&

# 2. Uygulamanın tamamen ayağa kalkması için yeterli süre bekle
time.sleep(7) # Bekleme süresi 7 saniyeye çıkarıldı

# 3. Ngrok tünelini aç
public_url = ngrok.connect(8501)

print("-" * 50)
print("Streamlit uygulaması ngrok üzerinden BAŞARILI ŞEKİLDE başlatıldı.")
print("Lütfen aşağıdaki External URL linkine tıklayın:")
print(f"WEB PROJENİZİN ADRESİ: {public_url}")
print("-" * 50)

--------------------------------------------------
Streamlit uygulaması ngrok üzerinden BAŞARILI ŞEKİLDE başlatıldı.
Lütfen aşağıdaki External URL linkine tıklayın:
WEB PROJENİZİN ADRESİ: NgrokTunnel: "https://meningeal-darin-unreprobatively.ngrok-free.dev" -> "http://localhost:8501"
--------------------------------------------------
